In [1]:
import pandas as pd
data = pd.read_csv('data\Topics.csv')

In [2]:
data['Title'] = data['Title'].str.normalize('NFKD').str.encode('ascii', errors = 'ignore').str.decode('utf-8')

In [3]:
def recommend_topic(film, number):
    if film == 'Pick a film':
        pass
    elif film == '':
        pass
    elif data['Title'].str.match(film, case = False).any():
        topic = int(data[data['Title'].str.match(film, case = False)]['Topic'].iloc[0])
        film_title = data[data['Title'].str.match(film, case = False)]['Title'].iloc[0]
        top_recommendations = data[data['Topic'] == topic].sort_values('averageRating', ascending = False)
        try: 
            top_recommendations = top_recommendations[~top_recommendations.Title.str.contains(film, case = False)][:number]
            print('Your film: ' + film_title)
            print('-' * 10)
            print('#' + str(number) + ' recommendation:')
            print('Title: ' + top_recommendations.iloc[number -1]['Title'])
            print('Director: ' + top_recommendations.iloc[number -1]['Director'])
            print('Country: ' + top_recommendations.iloc[number -1]['Country'])
            print('Year: ' + str(top_recommendations.iloc[number -1]['Year']))
            print('Duration: ' + top_recommendations.iloc[number -1]['Duration'])
            print('Film description: ' + top_recommendations.iloc[number -1]['Description'])
            print('Topic description: '+ top_recommendations.iloc[number -1]['Topic Description'])
            print('IMDB average rating: ' + str(top_recommendations.iloc[number -1]['averageRating']))
            print('IMDB number of votes: ' + str(top_recommendations.iloc[number -1]['numVotes']))
            print('Url: ' + top_recommendations.iloc[number -1]['Url'])
        except: 
            print('No more recommendations are available')
    else:
        print('Film not found')

### Recommendations based on the highest rated films within target film's topic

In [4]:
from ipywidgets import interactive
import ipywidgets as widgets
interactive(recommend_topic,
           film = widgets.Text(description = 'Film', value = 'Pick a film'),
           number = widgets.BoundedIntText(description = 'Rec #', min = 1)
           )

interactive(children=(Text(value='Pick a film', description='Film'), BoundedIntText(value=1, description='Rec …

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words = 'english' )
tfidf_matrix = tfidf.fit_transform(data['Description'])
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
indices = pd.Series(data.index, index = data['Title']).drop_duplicates()

In [6]:
def recommend_cosine_sim(film, number):
    if film == 'Pick a film':
        pass
    elif film == '':
        pass
    elif data['Title'].str.match(film, case = False).any():
        film_title = data[data['Title'].str.match(film, case = False)]['Title'].iloc[0]
        idx = indices[film_title]
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:11]
        movie_indices = [i[0] for i in sim_scores]
        print('Your film: ' + film_title)
        print('-' * 10)
        print('#' + str(number) + ' recommendation:')
        print('Title: ' + data.loc[movie_indices[number - 1]]['Title'])
        print('Director: ' + data.loc[movie_indices[number - 1]]['Director'])
        print('Country: ' + data.loc[movie_indices[number - 1]]['Country'])
        print('Year: ' + str(data.iloc[number -1]['Year']))
        print('Duration: ' + data.loc[movie_indices[number - 1]]['Duration'])
        print('Description: ' + data.loc[movie_indices[number - 1]]['Description'])
        print('IMDB average rating: ' + str(data.loc[movie_indices[number - 1]]['averageRating']))
        print('IMDB number of votes: ' + str(data.loc[movie_indices[number - 1]]['numVotes']))
        print('Url: ' + data.loc[movie_indices[number - 1]]['Url'])
    else:
        print('Film not found')

### Recommendations based on highest cosine similarity to target film

In [7]:
from ipywidgets import interactive
import ipywidgets as widgets
interactive(recommend_cosine_sim,
           film = widgets.Text(description = 'Film', value = 'Pick a film'),
           number = widgets.BoundedIntText(description = 'Rec #', min = 1, max = 10)
           )

interactive(children=(Text(value='Pick a film', description='Film'), BoundedIntText(value=1, description='Rec …